# Introduction to CUDA and PyCUDA

In [ ]:
#!pip install pycuda # install cuda
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [ ]:
import numpy
N = 1000

#// wrapper para checar erros nas chamadas de funções de CUDA
##define CUDA_SAFE_CALL(call){ \
#  cudaError_t err = call; \
#  if(err != cudaSuccess){ \
#    fprintf(stderr,"Erro no arquivo '%s', linha %i: %s.\n", __FILE__, __LINE__, cudaGetErrorString(err)); \
#    exit(EXIT_FAILURE); \
#  }\
#}

# Definição do kernel
mod = SourceModule("""
  __global__ void VecAdd(float *a, float *b, float *c)
  {
    int idx = threadIdx.x;
    c[idx] = a[idx] + b[idx];
  }
  """)

# inicializa os vetores a e b
a = numpy.array([i for i in range(1, N+1)])
b = numpy.array([i for i in range(1, N+1)])
c = numpy.zeros(N)

a = a.astype(numpy.float32)
b = b.astype(numpy.float32)
c = c.astype(numpy.float32)

# aloca espaço para os vetores na GPU
a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)

# copia os vetores da CPU para a GPU (host para device)
cuda.memcpy_htod(a_gpu, a)
cuda.memcpy_htod(b_gpu, b)
cuda.memcpy_htod(c_gpu, c)

# Dispara o kernel com N threads
func = mod.get_function("VecAdd")
func(a_gpu, b_gpu, c_gpu, block=(N,1,1))

# copia resultado da GPU para a CPU (device para host)
c_result = numpy.empty_like(a)
cuda.memcpy_dtoh(c_result, c_gpu)

print(False not in (a+b == c_result))

True
